In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

타이타닉 훈련셋과 테스트셋 불러오기

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

훈련셋 정보 확인

In [ ]:
train.head()

In [ ]:
test.head()

Survived: 생존 여부 / 0 = No, 1 = Yes

Pclass: 티켓 등급 / 1 = 1st, 2 = 2nd, 3 = 3rd

Sex: 성별

Age: 나이

Sibsp: 함께 탑승한 형제자매, 배우자의 수

Parch: 함께 탑승한 부모, 자식의 수

Ticket: 티켓 번호

Fare: 티켓 비용

Cabin: 객실 번호

Embarked: 선착장 [ C = Cherbourg, Q = Queenstown, S = Southampton]

In [ ]:
train.shape

In [ ]:
test.shape

Age, Cabin 에서 몇개의 데이터가 빠져있는걸 확인 가능

In [ ]:
train.info()

In [ ]:
test.info()

훈련셋에는 Age : 177개 Canin : 687개 값들이 빠져있다.

In [ ]:
train.isnull().sum()

테스트셋에서도 마찬가지로 Age와 Cabin에 값들이 빠져있다.

In [ ]:
test.isnull().sum()

우선, Age, Cabin, Embarked 속성에 결측치가 있음을 확인했다. Cabin은 결측치의 비율이 전체의 77%이므로 무시하는것으로 한다. Age의 경우 중간값으로 결측치를 채우기로 하자.

Name과 Ticket 속성은 모델이 사용할 수 있는 유용한 숫자로 변환하기 어렵기 때문에 이 특성 또한 무시한다.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

성별, 나이, 자리 등급 등 특성에 따라 죽었는지 살았는지 그림으로 나타내기

In [ ]:
def pie_chart(feature):
    feature_ratio = train[feature].value_counts(sort=False)
    feature_size = feature_ratio.size
    feature_index = feature_ratio.index
    survived = train[train['Survived'] == 1][feature].value_counts()
    dead = train[train['Survived'] == 0][feature].value_counts()  
    
    plt.plot(aspect='auto')
    plt.pie(feature_ratio, labels=feature_index, autopct='%1.1f%%')
    plt.title(feature + '\'s ratio in total')
    plt.show()

    for i, index in enumerate(feature_index):
        plt.subplot(1, feature_size + 1, i + 1, aspect='equal')
        plt.pie([survived[index], dead[index]], labels=['Survivied', 'Dead'], autopct='%1.1f%%')
        plt.title(str(index) + '\'s ratio')
        
    plt.show()

In [ ]:
def bar_chart(feature):
    survived = train[train['Survived']==1][feature].value_counts()
    dead = train[train['Survived']==0][feature].value_counts()
    df = pd.DataFrame([survived,dead])
    df.index = ['Survived','Dead']
    df.plot(kind='bar',stacked=True, figsize=(10,5))

In [ ]:
pie_chart('Sex')

남자 : 0 여자 : 1

남성이 여성보다 많이 탔으며 남성보다 여성의 생존비율이 높다는것을 볼 수 있다.

In [ ]:
bar_chart('Sex')

여자들이 남자들보다 살아날 확률이 높았다는걸 알 수 있다.

In [ ]:
pie_chart('Pclass')

위와 같이 Pclass가 3인 사람들의 수가 가장 많았으며, Pclass가 높을수록 생존 비율이 높다는 것을 알 수 있다.

In [ ]:
bar_chart('Pclass')

1등급 클래스가 생존확률에 큰 역할을 했다는 것을 알 수 있다.

In [ ]:
bar_chart('SibSp')

타이타닉에 동승자에 따른 생존율과 사망율을 보여준다.

동승자가 있을경우 생존율이 좀 더 있다는 것을 보여준다.

In [ ]:
bar_chart('Parch')

혼자 탑승했을 때 보다 부모님이나 아이들과 탑승했을때 더 많이 산 것을 볼 수 있다.

In [ ]:
pie_chart('Embarked')

위와 같이 S에서 선착한 사람이 가장 많았으며, C에서 탄 사람 중에서는 생존한 사람의 비율이 높았고, 나머지 두 선착장에서 탄 사람들은 생존한 사람보다 그렇지 못한 사람이 조금 더 많았다.

지금까지 살펴본 데이터 특성들을 간략하게 종합해보면,

성별이 여성일 수록, 

Pclass가 높을 수록, 

C선착장에서 배를 탔다면,

형제, 자매, 배우자, 부모, 자녀와 함께 배에 탔다면,

생존 확률이 더 높았다는 것을 볼 수 있다.

# **데이터전처리**

수치형데이터 파이프라인

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler


num_pipeline = Pipeline([ ("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])

범주형데이터 파이프라인

In [ ]:
cat_pipeline = Pipeline([
        ("ordinal_encoder", OrdinalEncoder()),    
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

수치형과 범주형 특성의 파이프라인

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = ["Age", "SibSp", "Parch", "Fare"] #수치형
cat_attribs = ["Pclass", "Sex", "Embarked"] #범주형

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [ ]:
X_train = preprocess_pipeline.fit_transform(train)
X_train

In [ ]:
y_train = train["Survived"]

이제 분류기를 통해 학습할 준비가 되었다.

**RandomForestClassifier** 적용

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(X_train, y_train)

훈련시키고 테스트 세트를 예측

In [ ]:
X_test = preprocess_pipeline.transform(test)
y_pred = forest_clf.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

SVC 모델 적용

In [ ]:
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto")
svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
svm_scores.mean()

RandomForest보다 SVC모델의 점수가 더 높다.

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot([1]*10, svm_scores, ".")
plt.plot([2]*10, forest_scores, ".")
plt.boxplot([svm_scores, forest_scores], labels=("SVM", "Random Forest"))
plt.ylabel("Accuracy")
plt.show()

결과를 향상 시킬수 있는 방법은

* 교차 검증 및 그리드 검색을 사용하여 더 많은 모델을 비교하고 하이퍼 파라미터를 조정한다.
* 더 많은 특성을 조절한다. 예를들어
    * 수치형 특성을 범주형특성으로 바꾼다. 예를들어 서로 다른 나이대의 그룹들은 각 생존율이 다르게 나온다. 나이 버킷 범주를 만들고 나이 대신 그것을 사용하는 데 도움이 될 수 있다.
    * Sibsp 와 Parch를 그들의 합으로 교체한다.
    * 생존 특성과 잘 상관되는 이름 부분을 식별
    * 사용하지 않은 Cabin열 사용

한번 새로운 훈련셋과 테스트셋을 만들어 비교해보자

In [ ]:
train2 = pd.read_csv('/kaggle/input/titanic/train.csv')
test2 = pd.read_csv('/kaggle/input/titanic/test.csv') 

15살 차이를 기준으로 나이를 쪼개 평균을 낸다.

In [ ]:
train2["AgeBucket"] = train2["Age"] // 15 * 15
test2["AgeBucket"] = test2["Age"] // 15 * 15
train2[["AgeBucket", "Survived"]].groupby(['AgeBucket']).mean()

Sibsp와 Parch 를 합한다.

In [ ]:
train2["RelativesOnboard"] = train2["SibSp"] + train2["Parch"]
test2["RelativesOnboard"] = test2["SibSp"] + test2["Parch"]
train2[["RelativesOnboard", "Survived"]].groupby(
    ['RelativesOnboard']).mean()

Survived특성과 잘 상관되는 이름 부분을 식별

In [ ]:
train2['Title'] = train2.Name.str.extract(' ([A-Za-z]+)\.') # ([A-Za-z]+).‘는 정규표현식인데, 공백으로 시작하고, .으로 끝나는 문자열을 추출할 때 저렇게 표현을 한다.
test2['Title'] = test2.Name.str.extract(' ([A-Za-z]+)\.')

추출한 Title을 가진 사람이 몇 명이 존재하는지 성별과 함께 표현을 해보자

In [ ]:
pd.crosstab(train2['Title'], train2['Sex'])

여기에서 흔하지 않은 Title은 Other로 대체하고 중복되는 표현을 통일하자.

In [ ]:
train2['Title'] = train2['Title'].replace(['Capt', 'Col', 'Countess', 'Don',
                                           'Dona', 'Dr', 'Jonkheer','Lady',
                                           'Major', 'Rev', 'Sir'], 'Other')
train2['Title'] = train2['Title'].replace('Mlle', 'Miss')
train2['Title'] = train2['Title'].replace('Mme', 'Mrs')
train2['Title'] = train2['Title'].replace('Ms', 'Miss')

train2[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

#테스트셋도 똑같이 적용
test2['Title'] = test2['Title'].replace(['Capt', 'Col', 'Countess', 'Don',
                                           'Dona', 'Dr', 'Jonkheer','Lady',
                                           'Major', 'Rev', 'Sir'], 'Other')
test2['Title'] = test2['Title'].replace('Mlle', 'Miss')
test2['Title'] = test2['Title'].replace('Mme', 'Mrs')
test2['Title'] = test2['Title'].replace('Ms', 'Miss')

그리고 추출한 Title 데이터를 학습하기 알맞게 String Data로 변형해주면 된다.

In [ ]:
train2['Title'] = train2['Title'].astype(str)
test2['Title'] = test2['Title'].astype(str)

이제 train2는 데이터 전처리를 통해 성능향상을 기대할 수 있는 훈련셋으로 변경되었다.

똑같이 같은 파이프라인을 통해 학습과 훈련을 시킨다.

In [ ]:
num_attribs2 = ["Parch", "Fare"] #수치형
cat_attribs2 = ["Pclass", "Sex", "Embarked", "AgeBucket", "Title"] #범주형

preprocess_pipeline2 = ColumnTransformer([
        ("num", num_pipeline, num_attribs2),
        ("cat", cat_pipeline, cat_attribs2),
    ])

In [ ]:
X_train2 = preprocess_pipeline2.fit_transform(train2)
X_train2

In [ ]:
y_train2 = train2["Survived"]

# **Train2의 모델 훈련**

**RandomForestClassifier**적용

In [ ]:
X_test2 = preprocess_pipeline2.transform(test2)

In [ ]:
svm_clf.fit(X_train2, y_train2)

In [ ]:
forest_scores2 = cross_val_score(forest_clf, X_train2, y_train2, cv=10)
forest_scores2.mean()

SVC 적용